In [1]:
"""
Sequential Bulk Email Client for Flask API
Sends emails one by one with retry logic and failure recovery
"""

import requests
import time
from typing import List, Dict
from dataclasses import dataclass
import json


@dataclass
class EmailData:
    """Email data structure"""

    receiver_email: str
    subject: str
    body: str


@dataclass
class EmailResult:
    """Result of email sending attempt"""

    receiver_email: str
    subject: str
    success: bool
    message: str
    attempts: int


class BulkEmailClient:
    """Sequential bulk email client with retry logic"""

    def __init__(self, api_url: str, max_retries: int = 3, retry_delay: float = 2.0):
        """
        Initialize bulk email client

        Args:
            api_url: Base URL of your Flask API (e.g., "http://localhost:5000")
            max_retries: Maximum retry attempts per email
            retry_delay: Delay in seconds between retries
        """
        self.api_url = api_url.rstrip("/")
        self.endpoint = f"{self.api_url}/v2/send-email"
        self.max_retries = max_retries
        self.retry_delay = retry_delay

        self.successful_emails: List[EmailResult] = []
        self.failed_emails: List[EmailResult] = []

    def send_single_email(
        self, email: EmailData, current: int, total: int
    ) -> EmailResult:
        """
        Send single email with retry logic

        Args:
            email: EmailData object
            current: Current email index
            total: Total number of emails

        Returns:
            EmailResult object
        """
        for attempt in range(1, self.max_retries + 1):
            try:
                print(
                    f"\n[{current}/{total}] Sending to {email.receiver_email} "
                    f"(Attempt {attempt}/{self.max_retries})"
                )

                payload = {
                    "receiver_email": email.receiver_email,
                    "subject": email.subject,
                    "body": email.body,
                }

                response = requests.post(
                    self.endpoint,
                    json=payload,
                    headers={"Content-Type": "application/json"},
                    timeout=30,
                )

                result = response.json()

                if response.status_code == 200 and result.get("status"):
                    print(f"✓ SUCCESS: {email.receiver_email}")
                    return EmailResult(
                        receiver_email=email.receiver_email,
                        subject=email.subject,
                        success=True,
                        message=result.get("message", "Success"),
                        attempts=attempt,
                    )
                else:
                    error_msg = result.get("message", "Unknown error")
                    if attempt < self.max_retries:
                        print(f"⚠ FAILED: {error_msg}")
                        print(f"   Retrying in {self.retry_delay} seconds...")
                        time.sleep(self.retry_delay)
                    else:
                        print(f"✗ FINAL FAILURE: {error_msg}")
                        return EmailResult(
                            receiver_email=email.receiver_email,
                            subject=email.subject,
                            success=False,
                            message=f"Failed after {self.max_retries} attempts: {error_msg}",
                            attempts=attempt,
                        )

            except requests.exceptions.Timeout:
                error_msg = "Request timeout"
                if attempt < self.max_retries:
                    print(f"⚠ TIMEOUT: Retrying in {self.retry_delay} seconds...")
                    time.sleep(self.retry_delay)
                else:
                    print(f"✗ FINAL FAILURE: {error_msg}")
                    return EmailResult(
                        receiver_email=email.receiver_email,
                        subject=email.subject,
                        success=False,
                        message=error_msg,
                        attempts=attempt,
                    )

            except requests.exceptions.ConnectionError:
                error_msg = "Connection error - API not reachable"
                if attempt < self.max_retries:
                    print(
                        f"⚠ CONNECTION ERROR: Retrying in {self.retry_delay} seconds..."
                    )
                    time.sleep(self.retry_delay)
                else:
                    print(f"✗ FINAL FAILURE: {error_msg}")
                    return EmailResult(
                        receiver_email=email.receiver_email,
                        subject=email.subject,
                        success=False,
                        message=error_msg,
                        attempts=attempt,
                    )

            except Exception as e:
                error_msg = f"Exception: {str(e)}"
                if attempt < self.max_retries:
                    print(f"⚠ ERROR: {error_msg}")
                    print(f"   Retrying in {self.retry_delay} seconds...")
                    time.sleep(self.retry_delay)
                else:
                    print(f"✗ FINAL FAILURE: {error_msg}")
                    return EmailResult(
                        receiver_email=email.receiver_email,
                        subject=email.subject,
                        success=False,
                        message=error_msg,
                        attempts=attempt,
                    )

        return EmailResult(
            receiver_email=email.receiver_email,
            subject=email.subject,
            success=False,
            message="Unknown error",
            attempts=self.max_retries,
        )

    def send_bulk_emails(self, total_emails: int) -> Dict:
        """
        Generate report after sending bulk emails

        Args:
            total_emails: Total number of emails

        Returns:
            Dictionary with results and statistics
        """
        start_time = time.time()
        duration = time.time() - start_time

        report = {
            "total": total_emails,
            "successful": len(self.successful_emails),
            "failed": len(self.failed_emails),
            "duration_seconds": round(duration, 2),
            "rate_per_second": round(total_emails / duration, 2) if duration > 0 else 0,
            "successful_emails": [
                {
                    "email": r.receiver_email,
                    "subject": r.subject,
                    "attempts": r.attempts,
                }
                for r in self.successful_emails
            ],
            "failed_emails": [
                {
                    "email": r.receiver_email,
                    "subject": r.subject,
                    "message": r.message,
                    "attempts": r.attempts,
                }
                for r in self.failed_emails
            ],
        }

        self.print_report(report)
        return report

    def retry_failed_emails(self, previous_report: Dict = None) -> Dict:
        """
        Retry only the emails that failed

        Args:
            previous_report: Previous report dict (optional, uses internal failed list if None)

        Returns:
            New report dictionary
        """
        if previous_report:
            failed_email_data = [
                EmailData(
                    receiver_email=failed["email"],
                    subject=failed["subject"],
                    body="Retry: This email failed previously",
                )
                for failed in previous_report["failed_emails"]
            ]
        else:
            failed_email_data = [
                EmailData(
                    receiver_email=result.receiver_email,
                    subject=result.subject,
                    body="Retry: This email failed previously",
                )
                for result in self.failed_emails
            ]

        if not failed_email_data:
            print("\n✓ No failed emails to retry")
            return {"total": 0, "successful": 0, "failed": 0}

        print(f"\n=== RETRYING {len(failed_email_data)} FAILED EMAILS ===")
        print("=" * 60)

        self.successful_emails.clear()
        self.failed_emails.clear()

        # Send failed emails one by one
        for idx, email in enumerate(failed_email_data, 1):
            result = self.send_single_email(email, idx, len(failed_email_data))

            if result.success:
                self.successful_emails.append(result)
            else:
                self.failed_emails.append(result)

        # Generate report
        return self.send_bulk_emails(len(failed_email_data))

    def print_report(self, report: Dict):
        """Print detailed report"""
        print("\n" + "=" * 60)
        print("BULK EMAIL REPORT")
        print("=" * 60)

        if report["total"] > 0:
            print(f"Total Emails:      {report['total']}")
            print(
                f"✓ Successful:      {report['successful']} "
                f"({report['successful'] * 100 / report['total']:.1f}%)"
            )
            print(
                f"✗ Failed:          {report['failed']} "
                f"({report['failed'] * 100 / report['total']:.1f}%)"
            )
            print(f"Duration:          {report['duration_seconds']} seconds")
            print(f"Rate:              {report['rate_per_second']} emails/second")

            if report["failed_emails"]:
                print("\nFailed Emails:")
                for failed in report["failed_emails"]:
                    print(f"  ✗ {failed['email']} - {failed['message']}")
        else:
            print("No emails to send")

        print("=" * 60 + "\n")

    def save_report(self, report: Dict, filename: str = "email_report.json"):
        """Save report to JSON file"""
        with open(filename, "w") as f:
            json.dump(report, f, indent=2)
        print(f"✓ Report saved to {filename}")

In [2]:
sub = f"""{{Company_Name}} - Your competitors are already automating this"""
s = sub.format(Company_Name="DemoCo")

In [3]:
def send_formatted_email(First_Name, Company_Name):
    body = f"""
    <html>

<body style="font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
    <p>Hi <strong>{First_Name}</strong>,</p>

    <p>Congratulations on launching <strong>{Company_Name}</strong>! I noticed your recent registration and wanted to reach out before you make a costly mistake most new companies make.</p>

    <p><strong>Here's the reality:</strong> 68% of startups that delay automation in their first year spend <strong>3x more</strong> fixing inefficiencies later. Your competitors are already moving faster.</p>

    <p>I'm <strong>Shivam</strong>, Founder of <strong>AxiometryAI</strong> (formerly known as VectorAI) a Bangalore-based AI automation company that's scaled to <strong>$10K MRR</strong> by delivering results that matter. We've deployed solutions for companies like IcSoft, VectorAI, and Databricks, helping them:</p>

    <ul style="margin: 15px 0;">
        <li>✅ Automate operations and <strong>cut costs by 40-60%</strong></li>
        <li>✅ Build high-converting web solutions that turn visitors into customers</li>
        <li>✅ Deploy intelligent chatbots that handle customer queries <strong>24/7</strong></li>
    </ul>

    <p><strong style="color: #d9534f;">The problem?</strong> Every day without automation:</p>

    <ul style="margin: 15px 0;">
        <li>You're losing potential revenue to slower response times</li>
        <li>Your team is drowning in repetitive tasks</li>
        <li>Your competitors are scaling faster with AI</li>
    </ul>

    <p><strong style="color: #5cb85c;">The solution?</strong> A <strong>15-minute call</strong> where I'll show you:</p>

    <ul style="margin: 15px 0;">
        <li>Exactly where you're bleeding money (most founders don't see this)</li>
        <li>How companies in your industry are using AI to <strong>10x their efficiency</strong></li>
        <li>A custom automation roadmap for <strong>{Company_Name}</strong></li>
    </ul>

    <p><em>No pitch. No pressure. Just actionable insights you can use immediately.</em></p>

    <p style="margin: 20px 0;">
        <strong>📅 Book a call with me here →</strong>
        <a href="https://cal.com/shivambaldha/v1" style="color: #0066cc; text-decoration: none; font-weight: bold;">https://cal.com/shivambaldha/v1</a>
    </p>

    <p>Or reply with your availability, and I'll work around your schedule.</p>

    <p style="margin-top: 30px;">Best,<br>
        <strong>Shivam B</strong><br>
        Founder, Axiometry AI Pvt Ltd.<br>
        📍 Bangalore | 📧 <a href="mailto:shivam@axiometryai.com" style="color: #0066cc;">shivam@axiometryai.com</a> | 📞 9019249765
    </p>

    <p style="margin-top: 20px; padding: 15px; background-color: #f8f9fa; border-left: 4px solid #0066cc;">
        <strong>P.S.</strong> - We're currently onboarding only <strong>3 new clients</strong> this month. If you're serious about scaling <strong>{Company_Name}</strong> efficiently, let's talk before those spots fill up.
    </p>
</body>

</html>
    """
    return body.strip()

In [4]:
from pprint import pprint
b = send_formatted_email("Shivam", "MathCo")
print(b)

<html>

<body style="font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
    <p>Hi <strong>Shivam</strong>,</p>

    <p>Congratulations on launching <strong>MathCo</strong>! I noticed your recent registration and wanted to reach out before you make a costly mistake most new companies make.</p>

    <p><strong>Here's the reality:</strong> 68% of startups that delay automation in their first year spend <strong>3x more</strong> fixing inefficiencies later. Your competitors are already moving faster.</p>

    <p>I'm <strong>Shivam</strong>, Founder of <strong>AxiometryAI</strong> (formerly known as VectorAI) a Bangalore-based AI automation company that's scaled to <strong>$10K MRR</strong> by delivering results that matter. We've deployed solutions for companies like IcSoft, VectorAI, and Databricks, helping them:</p>

    <ul style="margin: 15px 0;">
        <li>✅ Automate operations and <strong>cut costs by 40-60%</strong></li>
        <li>✅ Build high-converting web solutio

In [5]:
# import pandas as pd
# df = pd.read_csv("./leads.csv")

In [6]:
# df_filter = df[["Company",'Director Name', 'Email']]
# df_filter

In [7]:
# emails = []

# for index, row in df_filter.iterrows():
#     company_name = row['Company'].title() if pd.notna(row['Company']) else "there"
#     first_name = row['Director Name'].title() if pd.notna(row['Director Name']) else "there"
#     receiver_email = row['Email']

#     personalized_subject = sub.format(Company_Name=company_name)
#     personalized_body = send_formatted_email(First_Name=first_name, Company_Name=company_name)
    
#     emails.append(EmailData(
#         receiver_email=receiver_email,
#         subject=personalized_subject,
#         body=personalized_body
#     ))

In [8]:
# type(emails[20])

In [9]:
# ============================================
# USAGE EXAMPLE
# ============================================
if __name__ == "__main__":
    # Initialize client
    client = BulkEmailClient(
        api_url="http://127.0.0.1:5000",  # Your Flask API URL
        max_retries=2,  # Retry each email 3 times if it fails
        retry_delay=2.0,  # Wait 2 seconds between retries
    )

    # Prepare bulk emails
    emails = [
        EmailData("helloshivam21@gmail.com", subject=s, body=b),
    ]

    print(f"\nStarting bulk email send: {len(emails)} emails")
    print("=" * 60)

    client.successful_emails.clear()
    client.failed_emails.clear()

    # Send emails one by one in sequence
    for idx, email in enumerate(emails, 1):
        result = client.send_single_email(email, idx, len(emails))

        if result.success:
            client.successful_emails.append(result)
        else:
            client.failed_emails.append(result)

    # Generate report
    report = client.send_bulk_emails(len(emails))

    # Save report
    client.save_report(report)

    # Retry failed emails if any
    if report["failed"] > 0:
        print("\n🔄 Retrying failed emails...")
        retry_report = client.retry_failed_emails(report)
        client.save_report(retry_report, "email_retry_report.json")
    else:
        print("\n🎉 All emails sent successfully!")


Starting bulk email send: 1 emails

[1/1] Sending to helloshivam21@gmail.com (Attempt 1/2)
✓ SUCCESS: helloshivam21@gmail.com

BULK EMAIL REPORT
Total Emails:      1
✓ Successful:      1 (100.0%)
✗ Failed:          0 (0.0%)
Duration:          0.0 seconds
Rate:              0 emails/second

✓ Report saved to email_report.json

🎉 All emails sent successfully!
